# Scrapbook

This notebook contains some examples of TimeAtlas usage

## Define a TimeSeries

The two essential components are a Pandas Series and a DatetimeIndex

In [1]:
from timeatlas import TimeSeries
from pandas import Series, DataFrame, DatetimeIndex
import pandas as pd

s = pd.read_csv("../src/timeatlas/data/dummy-bbdata/13336.csv")
s = DataFrame(data=s["value"].values, index=pd.to_datetime(s["timestamp"]).values)

In [2]:
s1 = s["2019-08-01":"2019-08-31"]
s2 = s["2019-09-01":"2019-09-30"]

In [3]:
ts = TimeSeries(s)
ts1 = TimeSeries(s1)
ts2 = TimeSeries(s2)

/Users/frederic.montet/.local/share/virtualenvs/timeatlas-6ip_oGBK/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [4]:
a,b = ts.split("2019-08-02")

In [11]:
index = DatetimeIndex(['2019-01-01', '2019-01-02',
                               '2019-01-03', '2019-01-04'])
my_series = pd.Series([0.4, 1.0, 0.7, 0.6], index=index)

In [19]:
my_df = pd.DataFrame([0.4, 1.0, 0.7, 0.6], index=index)

In [54]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
df = DataFrame({"one":my_series, "two":my_series})
df = df.reset_index()

In [55]:
df

,index,one,two
0,2019-01-01,0.4,0.4
1,2019-01-02,1.0,1.0
2,2019-01-03,0.7,0.7
3,2019-01-04,0.6,0.6


In [64]:
df1 = pd.DataFrame(df[["one", "two"]])
df1.index = df["index"]

In [65]:
df1

,one,two
index,,
2019-01-01,0.4,0.4
2019-01-02,1.0,1.0
2019-01-03,0.7,0.7
2019-01-04,0.6,0.6


In [25]:
my_df.columns = ["values"]

In [28]:
"values" in my_df.columns 

True

In [9]:
a.series.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [7]:
def add(x,y):
    return x + y

ts1.apply(add, ts2)

ValueError: If using all scalar values, you must pass an index

## Define a Metadata object

In [ ]:
from timeatlas import Metadata, types

# You can use some typed metadata object
my_unit = types.Unit("power", "W", "float")
my_sensor = types.Sensor(2902, "HB/floor2/22-23C/Prises_Power_Tot")

# Or use Python dictionaries
my_location = {
    "building" : "Blue Factory",
    "floor" : "12",
    "room"  : "22C"
}
my_coordinates = {
    "lat" : 46.796611,
    "lon" : 7.147563
}
my_dict = {
    "unit": my_unit,
    "sensor": my_sensor,
    "location": my_location,
    "coordinates": my_coordinates
}

# Create the Metadata object
my_meta = Metadata(my_dict)


## Define a TimeSeries with Metadata

In [ ]:
ts_meta = TimeSeries(s, my_meta)
ts_meta

## Make Predictions

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
from timeatlas import models

In [ ]:
m1 = models.Prophet()
m1.fit(ts_meta)
pts1 = m1.predict(ts_meta)
pts2 = m1.predict("24 hours")

In [ ]:
m2 = models.LinearRegression()
m2.fit(ts_meta)
pts3 = m2.predict(ts_meta)
pts4 = m2.predict("14 days")

In [ ]:
from timeatlas import plots

plots.prediction(ts_meta, pts1)

## Detect Anomalies


In [ ]:
from timeatlas import detectors, metrics

In [ ]:
ts.apply(lambda x: x+1).series.head()

In [ ]:
# Create the base of the detector. 
# i.e. this detector is based on this model and compute this error
d = detectors.Surprise(m1, metrics.relative_error)

# Set the alerts
d.alerts("quantile", [0.65, 0.8, 0.85, 0.95])\

# Fit the detector with a specific Time Series
d.fit(ts_meta)

In [ ]:
# Detect anomalies with the fitted detector
a = d.detect(ts2)

In [ ]:
a.plot()

In [ ]:
d.truth.plot()
d.prediction.plot()

## Scalers

In [ ]:
from timeatlas.processing import scalers

In [ ]:
ts_meta.plot()

In [ ]:
y = scalers.minmax(ts_meta)

In [ ]:
y.plot()